# langchain

In [1]:
!pip install -q torch transformers transformers accelerate bitsandbytes langchain sentence-transformers faiss-gpu openpyxl pacmap datasets langchain-community ragatouille

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 647.5/647.5 kB 34.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 7.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.6/50.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 53.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 38.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 81.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.7/41.7 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
corpus_path = '/content/drive/MyDrive/11711AdvancedNLP/rag-assignment/data'
chunk_size = 512
chunk_overlap = 128
index_name = "faiss_index_1025"

EMBEDDING_MODEL_NAME = "BAAI/bge-m3"
# READER_MODEL_NAME = "facebook/llama2-7B"

q_path = '/content/drive/MyDrive/11711AdvancedNLP/rag-assignment/qa_pairs_new/zzx_questions.txt' # Update with the path to your Qs
actual_a_path = '/content/drive/MyDrive/11711AdvancedNLP/rag-assignment/actual_answer.txt'  # Update with the path to your actual output file
reference_a_path = '/content/drive/MyDrive/11711AdvancedNLP/rag-assignment/qa_pairs_new/zzx_answers.txt'  # Update with the path to your reference output file



# read all txt file and combine into a Langchain Document

In [ ]:
import os
from langchain.docstore.document import Document as LangchainDocument
from tqdm import tqdm  # for progress bar

# Step 1: Specify the directory containing the text files
folder_path = corpus_path
docs = []

# Step 2: Read all text files from the folder
for file_name in os.listdir(folder_path):
    # Only process .txt files
    if file_name.endswith('.txt'):
        file_path = os.path.join(folder_path, file_name)
        with open(file_path, 'r') as f:
            content = f.read()
            # Create a dictionary with text and source
            docs.append({"text": content, "source": file_name})

# Step 3: Convert to LangchainDocument objects
RAW_KNOWLEDGE_BASE = [
    LangchainDocument(page_content=doc["text"], metadata={"source": doc["source"]}) for doc in tqdm(docs)
]

# RAW_KNOWLEDGE_BASE now contains your converted documents from all .txt files in the 'data' folder


100%|██████████| 28/28 [00:00<00:00, 8556.69it/s]


# Separate the documents into chunks

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from typing import Optional, List, Tuple

from transformers import AutoTokenizer

MARKDOWN_SEPARATORS = [
    "\n#{1,6} ",
    "```\n",
    "\n\\*\\*\\*+\n",
    "\n---+\n",
    "\n___+\n",
    "\n\n",
    "\n",
    " ",
    "",
]

def split_documents(
    chunk_size: int,
    knowledge_base: List[LangchainDocument],
    tokenizer_name: Optional[str] = EMBEDDING_MODEL_NAME,
) -> List[LangchainDocument]:
    """
    Split documents into chunks of maximum size `chunk_size` tokens and return a list of documents.
    """
    text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(
        AutoTokenizer.from_pretrained(tokenizer_name),
        chunk_size = chunk_size,  # The maximum number of characters in a chunk: we selected this value arbitrarily
        chunk_overlap = chunk_overlap,
        add_start_index=True,
        strip_whitespace=True,
        separators=MARKDOWN_SEPARATORS,
    )

    docs_processed = []
    for doc in knowledge_base:
        docs_processed += text_splitter.split_documents([doc])

    # Remove duplicates
    unique_texts = {}
    docs_processed_unique = []
    for doc in docs_processed:
        if doc.page_content not in unique_texts:
            unique_texts[doc.page_content] = True
            docs_processed_unique.append(doc)

    return docs_processed_unique


docs_processed = split_documents(
    chunk_size,  # We choose a chunk size adapted to our model
    RAW_KNOWLEDGE_BASE,
    tokenizer_name=EMBEDDING_MODEL_NAME,
)

# Let's visualize the chunk sizes we would have in tokens from a common model
# from transformers import AutoTokenizer

# tokenizer = AutoTokenizer.from_pretrained(EMBEDDING_MODEL_NAME)
# lengths = [len(tokenizer.encode(doc.page_content)) for doc in tqdm(docs_processed)]
# fig = pd.Series(lengths).hist()
# plt.title("Distribution of document lengths in the knowledge base (in count of tokens)")
# plt.show()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (14523 > 8192). Running this sequence through the model will result in indexing errors


# Generate embeddings and create Index in FAISS.from_documents()

In [7]:
from langchain.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores.utils import DistanceStrategy

embedding_model = HuggingFaceEmbeddings(
    model_name=EMBEDDING_MODEL_NAME,
    multi_process=True,
    model_kwargs={"device": "cuda", "trust_remote_code": True},
    encode_kwargs={"normalize_embeddings": True},  # Set `True` for cosine similarity
)

# KNOWLEDGE_VECTOR_DATABASE = FAISS.from_documents(
#     docs_processed, embedding_model, distance_strategy=DistanceStrategy.COSINE
# )

# KNOWLEDGE_VECTOR_DATABASE.save_local(index_name)

<ipython-input-7-a060424eb465>:5: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [ ]:
!zip -r faiss_index.zip "/content/faiss_index_1025"

  adding: content/faiss_index_1025/ (stored 0%)
  adding: content/faiss_index_1025/index.pkl (deflated 75%)
  adding: content/faiss_index_1025/index.faiss (deflated 7%)


In [5]:
!unzip /content/drive/MyDrive/11711AdvancedNLP/rag-assignment/faiss_index-2.zip

Archive:  /content/drive/MyDrive/11711AdvancedNLP/rag-assignment/faiss_index-2.zip
   creating: content/faiss_index_1025/
  inflating: content/faiss_index_1025/index.pkl  
  inflating: content/faiss_index_1025/index.faiss  


In [8]:
KNOWLEDGE_VECTOR_DATABASE = FAISS.load_local(
    "/content/drive/MyDrive/11711AdvancedNLP/rag-assignment/faiss_index_1025", embedding_model, allow_dangerous_deserialization=True
)

# create the reader, format the prompt

## Llama2-7B

In [9]:
from huggingface_hub import notebook_login

# Log in to Hugging Face in the notebook to retrieve your token
notebook_login()

In [10]:
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

torch_dtype=torch.float16

# Assuming llama2-7B is available under this name in the Hugging Face Model Hub
model_name = "meta-llama/Llama-2-7b-chat-hf"  # Update with the correct model path if different


bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(model_name, quantization_config=bnb_config)
# Load the model and tokenizer for LLaMA 2-7B
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Create a pipeline for text-generation using LLaMA 2-7B
READER_LLM = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.5,
    top_k=10,
    top_p=0.8,
    return_full_text=False,
    max_new_tokens=50,
    num_beams=3,
)

# Prepare the prompt format for usage with the chat template
prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, generate answer based on your own knowledge.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
# Assuming you want to apply a chat template, make sure LLaMA supports or configure it accordingly
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)

config.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.62k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Zephyr

In [ ]:
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)
model = AutoModelForCausalLM.from_pretrained(READER_MODEL_NAME, quantization_config=bnb_config)
tokenizer = AutoTokenizer.from_pretrained(READER_MODEL_NAME)

READER_LLM = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.5,
    top_k=10,
    top_p=0.8,
    return_full_text=False,
    max_new_tokens=50,
    num_beams=3,
)

prompt_in_chat_format = [
    {
        "role": "system",
        "content": """Using the information contained in the context,
give a comprehensive answer to the question.
Respond only to the question asked, response should be concise and relevant to the question.
Provide the number of the source document when relevant.
If the answer cannot be deduced from the context, generate answer based on your own knowledge.""",
    },
    {
        "role": "user",
        "content": """Context:
{context}
---
Now here is the question you need to answer.

Question: {question}""",
    },
]
RAG_PROMPT_TEMPLATE = tokenizer.apply_chat_template(
    prompt_in_chat_format, tokenize=False, add_generation_prompt=True
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

# Load QA pairs, retrieve top-k chunks, generate answers


## w/o reranker

In [11]:
import os

# Folder where the text files with questions are located

# Initialize an empty list to store the questions
questions = []



# Step 1: Read all text files from the folder

with open(q_path, 'r') as f:
    # Read the entire content of the file as a question
    question = f.read().strip()
    question = question.split('\n')
    questions.append(question)

# Initialize an empty list to store the answers
answers = []

# Step 2: Loop through each question in the list and generate answers
for question in questions[0]:
    # query_vector = embedding_model.embed_query(question)
    retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=question, k=2)

    retrieved_docs_text = [doc.page_content for doc in retrieved_docs]  # We only need the text of the documents
    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(retrieved_docs_text)])
    # Format the final prompt with the current question
    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Generate the answer using the reader model
    answer = READER_LLM(final_prompt)[0]["generated_text"]

    # Append the answer to the answers list
    answers.append(answer)
    print(f"Q: {question}.\nA: {answer}\n")
# Step 3: Print all the questions and their corresponding answers
# for question, answer in zip(questions, answers):
#     print(f"Answer: {answer}\n")
# Step 3: Write the answers to the file, each on a new line
with open(actual_a_path, 'w') as f:
    for answer in answers:
        f.write(answer + '\n')

print(f"Answers have been written to {actual_a_path}.")

Q: What is the significance of 1783 in the history of Pittsburgh?.
A:   Based on the information provided in the context, 1783 is a significant year in the history of Pittsburgh as it marks the end of the American Revolutionary War and the signing of the Treaty of Paris, which recognized the independence

Q: What types of exhibits are available at the Carnegie Museum of Natural History?.
A:   Based on the information provided in the context, the Carnegie Museum of Natural History offers a wide range of exhibits, including:

1. Dinosaurs: The museum houses a collection of 230 dinosaur foss

Q: What are the core values of the Pittsburgh Bureau of Police?.
A:   Based on the information provided in the context, the core values of the Pittsburgh Bureau of Police are:

1. Respect
2. Trustworthiness
3. Neutrality
4. Transparency

Th

Q: When was the Andy Warhol Museum in Pittsburgh established?.
A:   Based on the information provided in the context, the Andy Warhol Museum in Pittsburgh was es

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Q: Who is the focus of the August Wilson Center for African American Culture?.
A:   Based on the information provided in the context, the focus of the August Wilson Center for African American Culture is August Wilson, a Pulitzer Prize-winning Pittsburgh playwright. According to Document 0, the park was redesigned

Q: Who established the first guidelines for the Pittsburgh Bureau of Police?.
A:   Based on the information provided in the context, the answer to the question "Who established the first guidelines for the Pittsburgh Bureau of Police?" is:

The first guidelines for the Pittsburgh Bureau of Police were established in

Q: Where can one learn about Pittsburgh's role during the French and Indian War?.
A:   According to Document 1, one can learn about Pittsburgh's role during the French and Indian War at the Fort Pitt Museum, which is located within Point State Park in Downtown Pittsburgh. The museum tells the story of Western Pennsylvania

Q: Where is the Children's Museum of Pi

## w/ reranker

In [14]:
import os
from transformers import AutoModelForSequenceClassification, AutoTokenizer

# Initialize the tokenizer and model for BGE reranking
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-v2-m3')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-v2-m3')
model.eval()
# Folder where the text files with questions are located

# Initialize an empty list to store the questions
questions = []



# Step 1: Read all text files from the folder

with open(q_path, 'r') as f:
    # Read the entire content of the file as a question
    question = f.read().strip()
    question = question.split('\n')
    questions.append(question)

# Initialize an empty list to store the answers
answers = []

# Step 2: Loop through each question in the list and generate answers
for question in questions[0]:
    # query_vector = embedding_model.embed_query(question)
    retrieved_docs = KNOWLEDGE_VECTOR_DATABASE.similarity_search(query=question, k=5)

    retrieved_docs_text = [doc.page_content for doc in retrieved_docs]  # We only need the text of the documents
    # Prepare pairs for reranking (question paired with each retrieved document)
    pairs = [[question, doc] for doc in retrieved_docs_text]
    # Rerank using BGE
    with torch.no_grad():
        inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
        scores = model(**inputs, return_dict=True).logits.view(-1, ).float()

    # Get indices of docs sorted by scores in descending order
    sorted_indices = torch.argsort(scores, descending=True)
    top_docs = [retrieved_docs_text[idx] for idx in sorted_indices[:2]] # top 2 ranked documents are selected

    context = "\nExtracted documents:\n"
    context += "".join([f"Document {str(i)}:::\n" + doc for i, doc in enumerate(top_docs)])
    # Format the final prompt with the current question
    final_prompt = RAG_PROMPT_TEMPLATE.format(question=question, context=context)

    # Generate the answer using the reader model
    answer = READER_LLM(final_prompt)[0]["generated_text"]

    # Append the answer to the answers list
    answers.append(answer)
    print(f"Q: {question}.\nA: {answer}\n")
# Step 3: Print all the questions and their corresponding answers
# for question, answer in zip(questions, answers):
#     print(f"Answer: {answer}\n")
# Step 3: Write the answers to the file, each on a new line
with open(actual_a_path, 'w') as f:
    for answer in answers:
        f.write(answer + '\n')

print(f"Answers have been written to {actual_a_path}.")

tokenizer_config.json:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/795 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

Q: What is the significance of 1783 in the history of Pittsburgh?.
A:   Based on the information provided in the context, 1783 is a significant year in the history of Pittsburgh because it marks the end of the Revolutionary War and the beginning of a new era for the city. After the war,

Q: What types of exhibits are available at the Carnegie Museum of Natural History?.
A:   Based on the information provided in the context, the Carnegie Museum of Natural History offers a wide range of exhibits, including:

1. Dinosaur fossils - The museum houses a collection of 230 dinosa

Q: What are the core values of the Pittsburgh Bureau of Police?.
A:   Based on the provided documents, the core values of the Pittsburgh Bureau of Police are:

1. Unbiased Policing
2. Integrity
3. Accountability
4. Respect
5. Community Involvement

Q: When was the Andy Warhol Museum in Pittsburgh established?.
A:   Based on the information provided in the context, the Andy Warhol Museum in Pittsburgh was established 

# Performance


In [16]:
from sklearn.metrics import precision_score, recall_score, f1_score
import numpy as np

def calculate_metrics(actual_a_path: str, reference_a_path: str):
    # Read the actual output file
    with open(actual_a_path, 'r') as f:
        actual_outputs = [line.strip() for line in f if line.strip()]

    # Read the reference output file
    with open(reference_a_path, 'r') as f:
        reference_outputs = [line.strip() for line in f if line.strip()]

    # Check if both files have the same number of lines
    if len(actual_outputs) != len(reference_outputs):
        raise ValueError("The number of lines in actual and reference files must be the same.")

    # Token-level comparison (binary classification of matching tokens)
    all_precisions, all_recalls, all_f1s = [], [], []

    for actual, reference in zip(actual_outputs, reference_outputs):
        # Tokenize the outputs
        actual_tokens = set(actual.split())
        reference_tokens = set(reference.split())

        # True positives, false positives, and false negatives
        true_positive = len(actual_tokens & reference_tokens)
        false_positive = len(actual_tokens - reference_tokens)
        false_negative = len(reference_tokens - actual_tokens)

        # Precision, Recall, and F1-score calculations
        precision = true_positive / (true_positive + false_positive) if (true_positive + false_positive) > 0 else 0.0
        recall = true_positive / (true_positive + false_negative) if (true_positive + false_negative) > 0 else 0.0
        f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

        # Store metrics for each example
        all_precisions.append(precision)
        all_recalls.append(recall)
        all_f1s.append(f1)

    # Calculate average metrics
    avg_precision = np.mean(all_precisions)
    avg_recall = np.mean(all_recalls)
    avg_f1 = np.mean(all_f1s)

    return avg_precision, avg_recall, avg_f1

# Example usage:

precision, recall, f1 = calculate_metrics(actual_a_path, reference_a_path)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")


Precision: 0.3713
Recall: 0.6345
F1-score: 0.4596
